In [1]:
import numpy as np

Some functions that we might want to use: 

In [2]:
def LCS(seq1, seq2):
    
    # base case 1
    if seq1 == "":
        return 0
    
    # base case 2
    elif seq2 == "":
        return 0
    
    # make a match while we can
    elif seq1[0] == seq2[0]:
        return 1 + LCS(seq1[1:], seq2[1:])
    
    # recurse on the two possible sub-problems
    else:
        lose1 = LCS(seq1, seq2[1:])
        lose2 = LCS(seq1[1:], seq2)
        
        return max(lose1, lose2)   

In [3]:
def differences(seq1, seq2):
    '''Counts the number of pairwise differences between
    two sequences'''
    
    count = 0
    
    for i in range(len(seq1)):
        if seq1[i] != seq2[i]:
            count += 1
            
    return count

The sequences we want to compare:

In [4]:
human =   "MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF.DLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR"
chimp =   "MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHF.DLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR"
gorilla = ".VLSPADKTNVKAAWGKVGAHAGDYGAEALERMFLSFPTTKTYFPHF.DLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR"
cow =     "MVLSAADKGNVKAAWGKVGGHAAEYGAEALERMFLSFPTTKTYFPHF.DLSHGSAQVKGHGAKVAAALTKAVEHLDDLPGALSELSDLHAHKLRVDPVNFKLLSHSLLVTLASHLPSDFTPAVHASLDKFLANVSTVLTSKYR"
horse =   "MVLSAADKTNVKAAWSKVGGHAGEYGAEALERMFLGFPTTKTYFPHF.DLSHGSAQVKAHGKKVGDALTLAVGHLDDLPGALSNLSDLHAHKLRVDPVNFKLLSHCLLSTLAVHLPNDFTPAVHASLDKFLSSVSTVLTSKYR"  
donkey =  "MVLSAADKTNVKAAWSKVGGNAGEFGAEALERMFLGFPTTKTYFPHF.DLSHGSAQVKAHGKKVGDALTLAVGHLDDLPGALSNLSDLHAHKLRVDPVNFKLLSHCLLSTLAVHLPNDFTPAVHASLDKFLSTVSTVLTSKYR" 
rabbit =  ".VLSPADKTNIKTAWEKIGSHGGEYGAEAVERMFLGFPTTKTYFPHF.DFTHGSZQIKAHGKKVSEALTKAVGHLDDLPGALSTLSDLHAHKLRVDPVNFKLLSHCLLVTLANHHPSEFTPAVHASLDKFLANVSTVLTSKYR"
carp =    "MSLSDKDKAAVKGLWAKISPKADDIGAEALGRMLTVYPQTKTYFAHWADLSPGSGPVKKHGKVIMGAVGDAVSKIDDLVGGLAALSELHAFKLRVDPANFKILAHNVIVVIGMLYPGDFPPEVHMSVDKFFQNLALALSEKYR"

In [5]:
animals = [carp, cow, donkey, horse, human, gorilla, rabbit]
nodes = ["carp", "cow", "donkey", "horse", "human", "gorilla", "rabbit"]

## Step 1: Make a dictionary of differences

In [6]:
def create_diff_dict(animals):
    '''Creates a matrix of differences between the given species.
    Stored in a dictionary of dictionary based on the indices of 
    the provided list.'''
    
    diff_dict = {}

    for i in range(len(animals)):
        for j in range(i+1, len(animals)):
            diff = differences(animals[i], animals[j])

            if(not i in diff_dict):
                diff_dict[i] = {}

            if(not j in diff_dict):
                diff_dict[j] = {}

            diff_dict[i][j] = diff
            diff_dict[j][i] = diff
        
    return diff_dict

In [7]:
create_diff_dict(animals)

{0: {1: 70, 2: 71, 3: 71, 4: 72, 5: 72, 6: 76},
 1: {0: 70, 2: 19, 3: 17, 4: 17, 5: 19, 6: 26},
 2: {0: 71, 1: 19, 3: 3, 4: 20, 5: 22, 6: 26},
 3: {0: 71, 1: 17, 2: 3, 4: 17, 5: 19, 6: 24},
 4: {0: 72, 1: 17, 2: 20, 3: 17, 5: 2, 6: 26},
 5: {0: 72, 1: 19, 2: 22, 3: 19, 4: 2, 6: 26},
 6: {0: 76, 1: 26, 2: 26, 3: 24, 4: 26, 5: 26}}

## Step 2: Write a function that finds the two closest related organisms (i.e. a pair group)


Your function should return the indices of the two most closely related organisms (i.e. 3 and 5). Make sure that it returns two unique values!

In [8]:
def pair_group(diff_dict):
    """ given a matrix of differences, returns the indices of the closest two related organisms"""
    
    min_val = np.inf
    index_1 = 0
    index_2 = 0
    
    for i in diff_dict.keys():
        for j in diff_dict[i].keys():
            if diff_dict[i][j] < min_val:
                min_val = diff_dict[i][j]
                index_1 = i
                index_2 = j
    
    return (index_1, index_2)

In [10]:
pair_group(create_diff_dict(animals))

(4, 5)

## Step 3: With a pair-group in hand, let's update our differences dictionary and nodes

In [21]:
def calculate_weight(tup):
    '''Determine number of items in a nested tuple'''
    if(type(tup) == int):
        return 1
    
    num_values = 0
    
    for i in tup:
        if type(i) == tuple:
            num_values += calculate_weight(i)
        else:
            num_values += 1
            
    return num_values

In [25]:
def generate_phylogenetic_tree(animals):
    '''Creates a phylogentic tree from a list of sequences'''
    
    # Create matrix of differences
    diff_dict = create_diff_dict(animals)

    # Apply UPGMA until all nodes have been combined
    while(len(diff_dict.keys()) > 1):
        # Get most related nodes
        (animal_1, animal_2) = pair_group(diff_dict)
        
        # Delete node pairs from dictionary
        diff_dict[animal_1].pop(animal_2)
        diff_dict[animal_2].pop(animal_1)

        # Add a new row for the new node
        new_node = (animal_1, animal_2)
        diff_dict[new_node] = {}

        # Calculate the weights for the arithmetic mean
        weight1 = calculate_weight(animal_1)
        weight2 = calculate_weight(animal_2)

        # Populate new node's row
        for i in diff_dict[animal_1].keys():
            diff_dict[new_node][i] = (weight1 * diff_dict[animal_1][i] + weight2 * diff_dict[animal_2][i]) / (weight1 + weight2)

        # Delete old nodes' rows
        diff_dict.pop(animal_1)
        diff_dict.pop(animal_2)

        # Update remaining nodes' distances to the newly added node and delete the remaining
        # references to the old nodes
        for i in diff_dict.keys():
            if(i != new_node):
                diff_dict[i][new_node] = (weight1 * diff_dict[i][animal_1] + weight2 * diff_dict[i][animal_2]) / (weight1 + weight2)
                
                diff_dict[i].pop(animal_1)
                diff_dict[i].pop(animal_2)
    
    # The last created node is the completed tree
    return new_node

In [26]:
def create_tree(tree, nodes):
    '''Generates a string based tree from a list of indices given a list of strings
    to map the indices to'''
    
    out = [0, 0]
    for i in range(len(tree)):
        if type(tree[i]) == tuple:
            out[i] = create_tree(tree[i], nodes)
        else:
            out[i] = nodes[tree[i]]
            
    return tuple(out)

In [27]:
index_tree = generate_phylogenetic_tree(animals)
create_tree(index_tree, nodes)

('carp', ('rabbit', (('donkey', 'horse'), ('cow', ('human', 'gorilla')))))

This tree gives the same result shown in class. The cow is placed closest to the human and gorilla, which is the correct placement.